In [35]:
import numpy as np
import utils
from sklearn.manifold import MDS
import torch
from torch import nn
import matplotlib.pyplot as plt
from autoencoders import AE, RegressionNN, train_loop, CombinedLoss, RegularizationLoss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from torch.nn.modules.loss import _Loss
from tqdm import tqdm, trange
from itertools import combinations

In [ ]:
class MDSAE(nn.Module):
    def __init__(self, n_input: int, latent_dim: int, resulting_dim: int, *decoder_args, train_frac: float=1.0, **decoder_kwargs):
        super(MDSAE, self).__init__()

        self.latent = nn.Parameter(torch.randn((n_input, latent_dim), dtype=torch.float32))
        self.n_train = int(n_input * train_frac)
        if train_frac == 1.0:
            self.valid = False
        else:
            self.valid = True
            

        if not ("act_fn" in decoder_kwargs.keys()):
            decoder_kwargs["act_fn"] = nn.Tanh
        self.decoder = RegressionNN(latent_dim, resulting_dim, *decoder_args, **decoder_kwargs)

    def forward(self, idx):
        train_outputs = self.decoder(self.latent[:self.n_train])
        if not self.valid:
            return train_outputs[idx.T]
        
        for param in self.decoder.parameters():
            param.requires_grad = False
        valid_outputs = self.decoder(self.latent[self.n_train:])
        for param in self.decoder.parameters():
            param.requires_grad = True

        outputs = torch.cat((train_outputs, valid_outputs), axis=0)
        return outputs[idx.T]
    

class MDSAELoss(_Loss):
    def __init__(self, dmat, lam: float = 1.0, val_lam: float=10.0, loss_function="default"):
        super(MDSAELoss, self).__init__()

        self.dmat = dmat
        self.lam = lam
        self.val_lam = val_lam
        if loss_function == "default":
            self.loss_function = nn.MSELoss()
        else:
            self.loss_function = loss_function

    def forward(self, model: MDSAE, inputs, outputs, targets):
        target_dists = self.dmat[inputs.T[0], inputs.T[1]]
        
        fst_mask, snd_mask = inputs.T < model.n_train

        train_valid_mask = fst_mask & torch.inverse(snd_mask)
        train_valid_outputs = outputs[train_valid_mask]
        train_valid_dists = compute_dists(train_valid_dists[0].detach(), train_valid_dists[1])
        train_valid_loss = self.val_lam * self.loss_function(train_valid_dists, target_dists[train_valid_mask])

        valid_mask = torch.inverse(fst_mask | snd_mask)
        valid_outputs = outputs[valid_mask]
        valid_dists = compute_dists(valid_outputs[0], valid_outputs[1])
        valid_loss = self.val_lam * self.loss_function(valid_dists, target_dists[valid_mask])
        
        if targets is None:
            # validation
            return self.lam * (train_valid_loss + valid_loss)
        
        train_mask = fst_mask & snd_mask
        train_outputs = outputs[train_mask]
        train_dists = compute_dists(train_outputs[0], train_outputs[1])
        train_loss = self.loss_function(train_dists, target_dists[train_mask])
        
        return self.lam * (train_loss + train_valid_loss + valid_loss)
    

def compute_dists(t1, t2):
    return torch.sqrt(((t1 - t2) ** 2).sum(axis=1))


def stress(dists, true_dists):
    return (((dists - true_dists) ** 2).sum() / dists ** 2).sum()

In [3]:
name = "coupled_oscillator"

data = np.load(f"trajectories/{name}.npz")["data"]
data = StandardScaler().fit_transform(np.concatenate(list(data))).reshape(data.shape)
dmat = utils.gen_dist_matrix(data)

Finished preprocessing 200 events in 0.0025s
   1990 / 19900  EMDs computed  -  10.00% completed - 2.291s
   3980 / 19900  EMDs computed  -  20.00% completed - 4.562s
   5970 / 19900  EMDs computed  -  30.00% completed - 6.856s
   7960 / 19900  EMDs computed  -  40.00% completed - 9.145s
   9950 / 19900  EMDs computed  -  50.00% completed - 11.295s
  11940 / 19900  EMDs computed  -  60.00% completed - 13.341s
  13930 / 19900  EMDs computed  -  70.00% completed - 15.395s
  15920 / 19900  EMDs computed  -  80.00% completed - 17.405s
  17910 / 19900  EMDs computed  -  90.00% completed - 19.340s
  19900 / 19900  EMDs computed  - 100.00% completed - 21.259s


In [4]:
data = torch.tensor(list(combinations(torch.arange(len(dmat)), 2)))
loss = CombinedLoss([MDSAELoss(torch.tensor(dmat, dtype=torch.float32)), RegularizationLoss()])
trained_models = []
for latent_dim in range(1, 10):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 10, 16, 2), data, data, 4, 1e-3, 20, loss, loss, valid_data=data, valid_target=data)
    trained_models.append(trained_model)

100%|███████████████████████████████████████████████████████████████| 20/20 [04:29<00:00, 13.49s/it]


Best loss: 11.35293960571289


100%|███████████████████████████████████████████████████████████████| 20/20 [04:37<00:00, 13.87s/it]


Best loss: 3.463139295578003


100%|███████████████████████████████████████████████████████████████| 20/20 [04:24<00:00, 13.23s/it]


...

Best loss: 1.2043405771255493


100%|███████████████████████████████████████████████████████████████| 20/20 [04:28<00:00, 13.41s/it]


Best loss: 1.192251443862915


100%|███████████████████████████████████████████████████████████████| 20/20 [04:25<00:00, 13.27s/it]

Best loss: 1.1382246017456055


In [5]:
data = torch.tensor(list(combinations(torch.arange(len(dmat)), 2)))
loss = CombinedLoss([MDSAELoss(torch.tensor(dmat, dtype=torch.float32)), RegularizationLoss()])
trained_models = []
for latent_dim in range(1, 6):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 10, 16, 2), data, data, 4, 1e-3, 100, loss, loss, valid_data=data, valid_target=data)
    trained_models.append(trained_model)

100%|█████████████████████████████████████████████████████████████| 100/100 [21:27<00:00, 12.88s/it]


Best loss: 5.270203590393066


100%|█████████████████████████████████████████████████████████████| 100/100 [21:21<00:00, 12.82s/it]


Best loss: 1.521048903465271


100%|█████████████████████████████████████████████████████████████| 100/100 [21:27<00:00, 12.88s/it]


Best loss: 1.175612211227417


100%|█████████████████████████████████████████████████████████████| 100/100 [21:29<00:00, 12.89s/it]


Best loss: 0.9873794317245483


100%|█████████████████████████████████████████████████████████████| 100/100 [22:38<00:00, 13.58s/it]

Best loss: 0.8704333305358887


In [9]:
samples = []
for model in trained_models:
    outputs = model(data).detach()
    dists = torch.sqrt(((outputs[0] - outputs[1]) ** 2).sum(axis=1))
    target_dists = torch.tensor(dmat, dtype=torch.float32)[data[:, 0], data[:, 1]]
    individual_losses = (dists - target_dists) ** 2
    samples.append(individual_losses.flatten())

In [11]:
from scipy.stats import ttest_ind


for i in range(4):
    print(ttest_ind(samples[i], samples[i + 1]).pvalue)

4.9054682121451244e-294
1.8522299866724198e-21
1.1550402577302254e-12
4.5432718238426245e-09


In [20]:
data = torch.tensor(list(combinations(torch.arange(len(dmat)), 2)))
loss = CombinedLoss([MDSAELoss(torch.tensor(dmat, dtype=torch.float32)), RegularizationLoss()])
more_trained_models = []
for latent_dim in range(1, 6):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 10, 16, 2), data, data, 256, 1e-3, 1000, loss, loss, valid_data=data, valid_target=data)
    more_trained_models.append(trained_model)

  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████| 1000/1000 [06:14<00:00,  2.67it/s]


Best loss: 4.863611698150635


100%|███████████████████████████████████████████████████████████| 1000/1000 [06:04<00:00,  2.75it/s]


Best loss: 1.4311840534210205


...

Best loss: 1.0970598459243774


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:50<00:00,  2.85it/s]


Best loss: 0.9211207628250122


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:35<00:00,  2.98it/s]

Best loss: 0.7528404593467712


In [22]:
samples = []
for model in more_trained_models:
    outputs = model(data).detach()
    dists = torch.sqrt(((outputs[0] - outputs[1]) ** 2).sum(axis=1))
    target_dists = torch.tensor(dmat, dtype=torch.float32)[data[:, 0], data[:, 1]]
    individual_losses = (dists - target_dists) ** 2
    samples.append(individual_losses.flatten())

for i in range(4):
    print(ttest_ind(samples[i], samples[i + 1]).pvalue)

1.4725249950086712e-277
1.5652374614941358e-23
2.2221136594570475e-13
1.0571708825695884e-22


In [24]:
loss = MDSAELoss(torch.tensor(dmat, dtype=torch.float32))
train_loss = CombinedLoss([loss, RegularizationLoss()])

collected_samples = []
for latent_dim in range(1, 6):
    print(f"latent dim: {latent_dim}")
    collected_samples.append([])
    for _ in range(20):
        trained_model = train_loop(MDSAE(len(dmat), latent_dim, 10, 16, 2), data, data, 256, 1e-3, 100, train_loss, loss, valid_data=data, valid_target=data)
        outputs = trained_model(data).detach()
        dists = torch.sqrt(((outputs[0] - outputs[1]) ** 2).sum(axis=1))
        target_dists = torch.tensor(dmat, dtype=torch.float32)[data[:, 0], data[:, 1]]
        individual_losses = (dists - target_dists) ** 2
        collected_samples[-1].append(individual_losses.flatten())
    print("---------------------------------------------------------\n\n")

latent dim: 1


100%|█████████████████████████████████████████████████████████████| 100/100 [00:36<00:00,  2.71it/s]


Best loss: 16.216371536254883


100%|█████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.82it/s]


Best loss: 11.744367599487305


...

Best loss: 2.457235097885132


100%|█████████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.05it/s]


Best loss: 3.243216037750244


100%|█████████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.07it/s]

Best loss: 3.6334786415100098
---------------------------------------------------------




In [31]:
# collected_samples = torch.cat([torch.cat(s) for s in collected_samples])
# print(collected_samples.mean(axis=2))
# print(collected_samples.mean(axis=(1, 2)))
# collected_samples.shape
# for d in collected_samples:


tensor([[16.2164, 11.7444, 11.5167, 11.6475, 12.6343, 16.5995, 18.6958, 17.4313,
         14.6669, 10.7451, 12.8488, 18.5324, 13.9004, 14.8215, 10.4894, 12.0246,
         17.7571, 34.5504, 14.8972,  9.9996],
        [ 6.6439, 13.2268, 10.2441,  5.7840,  6.1144,  8.2713,  7.7116,  5.8690,
          9.2178,  6.3677,  5.8065, 10.0600,  7.0071,  6.1526,  6.6495,  5.9070,
          7.2876,  6.7156,  7.2210,  5.9904],
        [ 3.7964,  4.4327,  3.9047,  4.2941,  5.0264,  4.3863,  4.6906,  4.1274,
          4.1244,  3.3257,  4.0801,  4.7396,  4.8877,  5.4255,  3.7585,  3.6666,
          4.1142,  3.9903,  4.2093,  4.8776],
        [ 3.5626,  2.4616,  3.2690,  2.5328,  3.0270,  3.3671,  4.0942,  3.0484,
          3.4911,  2.4044,  3.5902,  6.7062,  3.5248,  4.0814,  3.0072,  2.7242,
          3.5299,  3.4317,  3.6206,  3.4992],
        [ 3.0514,  2.3836,  2.2431,  2.7796,  2.8796,  2.3676,  2.1204,  2.6808,
          3.4782,  3.9391,  3.3729,  2.8574,  2.8009,  2.4719,  3.3788,  2.1045,
      

torch.Size([5, 20, 19900])

In [32]:
data = torch.tensor(list(combinations(torch.arange(len(dmat)), 2)))
loss = MDSAELoss(torch.tensor(dmat, dtype=torch.float32))
train_loss = CombinedLoss([loss, RegularizationLoss()])

more_trained_models = []
for latent_dim in range(1, 6):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 50, 32, 3), data, data, 256, 1e-3, 1000, train_loss, loss, valid_data=data, valid_target=data)
    more_trained_models.append(trained_model)

100%|███████████████████████████████████████████████████████████| 1000/1000 [06:10<00:00,  2.70it/s]


Best loss: 2.7746689319610596


100%|███████████████████████████████████████████████████████████| 1000/1000 [06:07<00:00,  2.72it/s]


Best loss: 0.4098174571990967


100%|███████████████████████████████████████████████████████████| 1000/1000 [06:06<00:00,  2.73it/s]


Best loss: 0.24046006798744202


100%|███████████████████████████████████████████████████████████| 1000/1000 [06:04<00:00,  2.74it/s]


Best loss: 0.183050736784935


100%|███████████████████████████████████████████████████████████| 1000/1000 [06:06<00:00,  2.73it/s]

Best loss: 0.16953767836093903


In [33]:
loss = MDSAELoss(torch.tensor(dmat, dtype=torch.float32))
train_loss = CombinedLoss([loss, RegularizationLoss()])

collected_samples_long = []
for latent_dim in range(1, 6):
    print(f"latent dim: {latent_dim}")
    collected_samples_long.append([])
    for _ in range(20):
        trained_model = train_loop(MDSAE(len(dmat), latent_dim, 10, 16, 2), data, data, 256, 1e-3, 1000, train_loss, loss, valid_data=data, valid_target=data)
        outputs = trained_model(data).detach()
        dists = torch.sqrt(((outputs[0] - outputs[1]) ** 2).sum(axis=1))
        target_dists = torch.tensor(dmat, dtype=torch.float32)[data[:, 0], data[:, 1]]
        individual_losses = (dists - target_dists) ** 2
        collected_samples_long[-1].append(individual_losses.flatten())
    print("---------------------------------------------------------\n\n")

latent dim: 1


AttributeError: 'Tensor' object has no attribute 'append'

In [34]:
even_more_trained_models = []
for latent_dim in range(1, 6):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 50, 128, 3), data, data, 256, 1e-3, 1000, train_loss, loss, valid_data=data, valid_target=data)
    even_more_trained_models.append(trained_model)

100%|███████████████████████████████████████████████████████████| 1000/1000 [12:45<00:00,  1.31it/s]


Best loss: 0.8698166608810425


100%|███████████████████████████████████████████████████████████| 1000/1000 [12:46<00:00,  1.30it/s]


Best loss: 0.06773122400045395


100%|███████████████████████████████████████████████████████████| 1000/1000 [11:19<00:00,  1.47it/s]


Best loss: 0.041964370757341385


100%|███████████████████████████████████████████████████████████| 1000/1000 [11:29<00:00,  1.45it/s]


Best loss: 0.03791968524456024


100%|███████████████████████████████████████████████████████████| 1000/1000 [10:19<00:00,  1.61it/s]

Best loss: 0.03550991788506508


In [38]:
embedding = MDS(n_components=50, max_iter=10000, n_init=100, dissimilarity="precomputed").fit_transform(dmat)
(((((embedding[data.T[0]] - embedding[data.T[1]]) ** 2).sum(axis=1) ** 0.5) - dmat[data.T[0], data.T[1]]) ** 2).mean()

/home/antony/.local/lib/python3.8/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


0.049763678213432375

In [45]:
even_more_trained_models = []
for latent_dim in range(1, 6):
    trained_model = train_loop(MDSAE(len(dmat), latent_dim, 50, 128, 3, train_frac=0.8), data, data, 256, 1e-3, 1000, train_loss, loss, valid_data=data)
    even_more_trained_models.append(trained_model)

100%|███████████████████████████████████████████████████████████| 1000/1000 [09:52<00:00,  1.69it/s]


Best loss: 1.695013165473938


100%|███████████████████████████████████████████████████████████| 1000/1000 [09:48<00:00,  1.70it/s]


Best loss: 0.459995299577713


100%|███████████████████████████████████████████████████████████| 1000/1000 [11:35<00:00,  1.44it/s]


Best loss: 0.26043254137039185


 89%|█████████████████████████████████████████████████████▎      | 888/1000 [09:58<01:18,  1.43it/s]